This notebook has been done using Tensorflow 1.7

In [ ]:
%tensorflow_version 1.x

import numpy as np
import sys

import matplotlib.pyplot as plt
import scipy.stats as stats

import matplotlib.patches as mpatch

import time
import psutil

import tensorflow as tf

In [ ]:
import codes.rnn_tf as rnn_tf
import codes.rnn_elman as rnn_elman # Con ruido
import codes.rnn_xhcy as rnn_xhcy # Dual CPU + Mem (conectada a CPU)

# Functions

In [ ]:
def automata_minimization(transition_matrix, final_states):

  matrix_aux = np.concatenate((transition_matrix, final_states[:, None]), axis=1)
  matrix_aux_unique = np.unique(matrix_aux, axis=0)
  list_items = [[] for u in matrix_aux_unique]
  for i, item in enumerate(matrix_aux):
    index = np.argmax([(item == u).all() for u in matrix_aux_unique])
    list_items[index].append(i)

  def minimize(matrix, list_items):
    len_list_items_ini = len(list_items)
    
    for item in list_items:
      for n in item:
        matrix[:, :-1][matrix[:, :-1] == n] = item[0]

    matrix_aux_unique = np.unique(matrix_aux, axis=0)
    list_items = [[] for u in matrix]
    for i, item in enumerate(matrix_aux):
      index = np.argmax([(item == u).all() for u in matrix_aux_unique])
      list_items[index].append(i)

    list_items = list_items[:len(matrix_aux_unique)]


    len_list_items_fin = len(list_items)

    if len_list_items_ini > len_list_items_fin:
      return minimize(matrix, list_items)
    else:
      return matrix_aux_unique.astype(int), [item[0] for item in list_items]

  return minimize(matrix_aux, list_items)

# Run

In [ ]:
# Posible values: BxA, paridad, tomita1, tomita2, tomita3, tomita4, tomita5, tomita6, tomita7

problema = "BxA"

input_file = "files/train/" + problema + "_train_x.txt"
output_file = "files/train/" + problema + "_train_y.txt"
test_largas_input = "files/test/" + problema + "_cadenasLargas_x.txt"
test_largas_output = "files/test/" + problema + "_cadenasLargas_y.txt"
test_extremo_muchasA_input = "files/test/" + problema + "_extremo_muchasA_x.txt"
test_extremo_muchasA_output = "files/test/" + problema + "_extremo_muchasA_y.txt"
test_extremo_muchasB_input = "files/test/" + problema + "_extremo_muchasB_x.txt"
test_extremo_muchasB_output = "files/test/" + problema + "_extremo_muchasB_y.txt"
test_grande_input = "files/test/" + problema + "_testGrande_x.txt"
test_grande_output = "files/test/" + problema + "_testGrande_y.txt"

hidden_size = 20
cpu_size = 10
seq_len = 25 # 25

num_batches = 10
noise_level = 0.
learning_rate = 1e-2
l1reg = 0.
shock = 0.

symbols = ['$', 'a', 'b']

In [ ]:
data_dic, dim_i, dim_o, _ = rnn_tf.data_transform_many_to_many(num_batches, 
                                                   seq_len, 
                                                   input_file=input_file, 
                                                   output_file=output_file,
                                                   chars_x = symbols,
                                                   chars_y = ['0', '1'],
                                                   longitud=1000000000)

dataset_x_train = data_dic['dataset_x'] 
dataset_y_train = data_dic['dataset_y']

## RNN Elman

In [ ]:
rnn = rnn_elman.rnn_elman_tf(dim_i, 
                             hidden_size,
                             #cpu_size,
                             dim_o,
                             learning_rate,
                             seq_len=seq_len,
                             noise_level=noise_level,
                             num_batches=num_batches,
                             regularizationL1=l1reg, 
                             shock=shock,
                             clipvalue=5.0,
                             #optimizer_name='Adam',
                             factor_inicializacion=0.5
                             )

In [ ]:
loss_grafica = rnn_tf.train(rnn, 100, dataset_x_train, dataset_y_train, 
                            write_tensorboard=False, 
                            ruido_progresivo=True,
                            ruido_min = 1.0,
                            ruido_max = 1.0,
                            pendiente_ruido = 2.0,
                            l1reg_mem_progresivo=True,
                            l1reg_mem_min = 0.1,
                            l1reg_mem_max = 0.1,
                            pendiente_l1reg_mem = 10.0,
                            l1reg_cpu_progresivo=True,
                            l1reg_cpu_min = 0.0,
                            l1reg_cpu_max = 0.0,
                            pendiente_l1reg_cpu = 1.0);

In [ ]:
plt.plot(loss_grafica, '-')

In [ ]:
data_dic, dim_i, dim_o, _ = rnn_tf.data_transform_many_to_many(1, 
                                                   seq_len, 
                                                   input_file=input_file, 
                                                   output_file=output_file,
                                                   chars_x = symbols,
                                                   chars_y = ['0', '1'],
                                                   longitud = 100000000)
dataset_x_train, dataset_y_train = data_dic['dataset_x'], data_dic['dataset_y']

data_dic, _, _, _ = rnn_tf.data_transform_many_to_many(1, 
                                         seq_len,
                                         input_file=test_largas_input, 
                                         output_file=test_largas_output,
                                         chars_x = symbols,
                                         chars_y = ['0', '1'],
                                         longitud = 100000000)
dataset_x_long, dataset_y_long = data_dic['dataset_x'], data_dic['dataset_y']

test_largas2_input = "files/test/" + problema + "_cadenasLargas2_x.txt"
test_largas2_output = "files/test/" + problema + "_cadenasLargas2_y.txt"


data_dic, _, _, _ = rnn_tf.data_transform_many_to_many(1,
                                           seq_len,
                                           input_file=test_extremo_muchasA_input, 
                                           output_file=test_extremo_muchasA_output,
                                           chars_x = symbols,
                                           chars_y = ['0', '1'],
                                           longitud = 100000000)
dataset_x_allAs, dataset_y_allAs = data_dic['dataset_x'], data_dic['dataset_y']

data_dic, _, _, _ = rnn_tf.data_transform_many_to_many(1,
                                           seq_len,
                                           input_file=test_extremo_muchasB_input, 
                                           output_file=test_extremo_muchasB_output,
                                           chars_x = symbols,
                                           chars_y = ['0', '1'],
                                           longitud = 100000000)
dataset_x_allBs, dataset_y_allBs = data_dic['dataset_x'], data_dic['dataset_y']


target, pred, porcentaje_acierto = rnn_tf.test(rnn, dataset_x_train, dataset_y_train)
print ("TEST TRAIN: " + str(porcentaje_acierto) + "% accuracy")
prediccion, esperado, porcentaje_acierto = rnn_tf.test(rnn, dataset_x_long, dataset_y_long)
print ("TEST LONG: " + str(porcentaje_acierto) + "% accuracy")
prediccion, esperado, porcentaje_acierto = rnn_tf.test(rnn, dataset_x_allAs, dataset_y_allAs)
print ("TEST EXTREMO A: " + str(porcentaje_acierto) + "% accuracy")
prediccion, esperado, porcentaje_acierto = rnn_tf.test(rnn, dataset_x_allBs, dataset_y_allBs)
print ("TEST EXTREMO B: " + str(porcentaje_acierto) + "% accuracy")

## Dual RNN

In [ ]:
data_dic, dim_i, dim_o, _ = rnn_tf.data_transform_many_to_many(num_batches, 
                                                   seq_len, 
                                                   input_file=input_file, 
                                                   output_file=output_file,
                                                   chars_x = symbols,
                                                   chars_y = ['0', '1'],
                                                   longitud=1000000000)

dataset_x_train = data_dic['dataset_x'] 
dataset_y_train = data_dic['dataset_y']

In [ ]:
rnn = rnn_xhcy.rnn_xhcy(dim_i, 
                        hidden_size,
                        cpu_size,
                        dim_o,
                        learning_rate,
                        seq_len=seq_len,
                        noise_level=noise_level,
                        num_batches=num_batches,
                        regularizationL1=l1reg, 
                        shock=shock,
                        clipvalue=5.0,
                        #optimizer_name='Adam',
                        factor_inicializacion=0.5
                        )

In [ ]:
loss_grafica = rnn_tf.train(rnn, 100, dataset_x_train, dataset_y_train, 
                            write_tensorboard=False, 
                            ruido_progresivo=True,
                            ruido_min = 1.0,
                            ruido_max = 1.0,
                            pendiente_ruido = 2.0,
                            l1reg_mem_progresivo=True,
                            l1reg_mem_min = 0.1,
                            l1reg_mem_max = 0.1,
                            pendiente_l1reg_mem = 10.0,
                            l1reg_cpu_progresivo=True,
                            l1reg_cpu_min = 0.0,
                            l1reg_cpu_max = 0.0,
                            pendiente_l1reg_cpu = 1.0);

In [ ]:
plt.plot(loss_grafica, '-')

In [ ]:
data_dic, dim_i, dim_o, _ = rnn_tf.data_transform_many_to_many(1, 
                                                   seq_len, 
                                                   input_file=input_file, 
                                                   output_file=output_file,
                                                   chars_x = symbols,
                                                   chars_y = ['0', '1'],
                                                   longitud = 100000000)
dataset_x_train, dataset_y_train = data_dic['dataset_x'], data_dic['dataset_y']

data_dic, _, _, _ = rnn_tf.data_transform_many_to_many(1, 
                                         seq_len,
                                         input_file=test_largas_input, 
                                         output_file=test_largas_output,
                                         chars_x = symbols,
                                         chars_y = ['0', '1'],
                                         longitud = 100000000)
dataset_x_long, dataset_y_long = data_dic['dataset_x'], data_dic['dataset_y']

test_largas2_input = "files/test/" + problema + "_cadenasLargas2_x.txt"
test_largas2_output = "files/test/" + problema + "_cadenasLargas2_y.txt"


data_dic, _, _, _ = rnn_tf.data_transform_many_to_many(1,
                                           seq_len,
                                           input_file=test_extremo_muchasA_input, 
                                           output_file=test_extremo_muchasA_output,
                                           chars_x = symbols,
                                           chars_y = ['0', '1'],
                                           longitud = 100000000)
dataset_x_allAs, dataset_y_allAs = data_dic['dataset_x'], data_dic['dataset_y']

data_dic, _, _, _ = rnn_tf.data_transform_many_to_many(1,
                                           seq_len,
                                           input_file=test_extremo_muchasB_input, 
                                           output_file=test_extremo_muchasB_output,
                                           chars_x = symbols,
                                           chars_y = ['0', '1'],
                                           longitud = 100000000)
dataset_x_allBs, dataset_y_allBs = data_dic['dataset_x'], data_dic['dataset_y']


target, pred, porcentaje_acierto = rnn_tf.test(rnn, dataset_x_train, dataset_y_train)
print ("TEST TRAIN: " + str(porcentaje_acierto) + "% accuracy")
prediccion, esperado, porcentaje_acierto = rnn_tf.test(rnn, dataset_x_long, dataset_y_long)
print ("TEST LONG: " + str(porcentaje_acierto) + "% accuracy")
prediccion, esperado, porcentaje_acierto = rnn_tf.test(rnn, dataset_x_allAs, dataset_y_allAs)
print ("TEST EXTREMO A: " + str(porcentaje_acierto) + "% accuracy")
prediccion, esperado, porcentaje_acierto = rnn_tf.test(rnn, dataset_x_allBs, dataset_y_allBs)
print ("TEST EXTREMO B: " + str(porcentaje_acierto) + "% accuracy")